Predicting Investor-Owned Electric Utilities in the U.S.

CST383 Data Science

Professor Ergezer

Jesus Martinez Miranda,

Jorge Vazquez,

Walid Elgammal,

Zuhra Totakhail

6/9/2025


Data Exploration

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import graphviz
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score



# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7


In [ ]:
# Define packages you want to install
packages = ['numpy', 'pandas', 'matplotlib', 'seaborn']

# Loop through and install if missing
for package in packages:
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")


numpy is already installed.
pandas is already installed.
matplotlib is already installed.
seaborn is already installed.
